In [31]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [32]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-01-08 15:58:26--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  44.2MB/s    in 0.6s    

2024-01-08 15:58:27 (44.2 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [33]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [34]:
counts1 = df_ratings['user'].value_counts()
counts2 = df_ratings['isbn'].value_counts()

df_ratings_filtered = df_ratings[~df_ratings['user'].isin(counts1[counts1 < 200].index)]
df_ratings_filtered = df_ratings_filtered[~df_ratings_filtered['isbn'].isin(counts2[counts2 < 100].index)]

In [35]:
df_merged = pd.merge(right=df_ratings_filtered, left=df_books, on="isbn")
df_merged = df_merged.drop_duplicates(["title", "user"])

In [36]:
utility_matrix = df_merged.pivot(index='title', columns='user', values='rating').fillna(0)


In [37]:
matrix = utility_matrix.values
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [45]:
def get_recommends(book_title):
    if book_title not in utility_matrix.index:
        return [book_title, []]

    book_vector = utility_matrix.loc[book_title].values.reshape(1, -1)
    distances, indices = knn_model.kneighbors(book_vector, n_neighbors=6)

    recommended_books = []
    for i in range(1, len(indices.flatten())):
        similar_book_title = utility_matrix.index[indices.flatten()[i]]
        similar_book_distance = distances.flatten()[i]
        recommended_books.append([similar_book_title, similar_book_distance])

    recommended_books.sort(key=lambda x: x[1], reverse=True)

    return [book_title, recommended_books]

book_recommendations = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(book_recommendations)


['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Catch 22', 0.7939835], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Interview with the Vampire', 0.73450685], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]


In [46]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
